In [15]:
import numpy as np
from qutip import about, basis, rand_ket, tensor
from qutip_qip.operations import Measurement
from qutip_qip.operations.gates import gate_sequence_product
from qutip_qip.qasm import read_qasm, print_qasm

In [16]:
path = "swap.qasm"
qasm_file = open(path, "r")
# print(qasm_file.read())

In [17]:
qc = read_qasm(path, mode="qiskit", version="2.0")
gate_sequence_product(qc.propagators())

Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=CSR, isherm=True
Qobj data =
[[1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]]

In [18]:
print_qasm(qc)

// QASM 2.0 file generated by QuTiP

OPENQASM 2.0;
include "qelib1.inc";

qreg q[2];

cx q[1],q[0];
cx q[0],q[1];
cx q[1],q[0];


## Custom Gates

In [19]:
path = "swap_custom.qasm"
qasm_file = open(path, "r")
# print(qasm_file.read())

In [20]:
qc = read_qasm(path)

In [21]:
qc.run(tensor(basis(2, 0), basis(2, 1)))

Quantum object: dims=[[2, 2], [1]], shape=(4, 1), type='ket', dtype=Dense
Qobj data =
[[0.]
 [0.]
 [1.]
 [0.]]

## Measurements and Classical Control

In [22]:
path = "teleportation.qasm"
qasm_file = open(path, "r")
qasm_str = qasm_file.read()
# print(qasm_str)

In [23]:
teleportation = read_qasm(qasm_str, strmode=True)

In [24]:
state = tensor(rand_ket(2), basis(2, 0), basis(2, 0))

initial_measurement = Measurement("start", targets=[0])
_, initial_probabilities = initial_measurement.measurement_comp_basis(state)

state_final = teleportation.run(state)

final_measurement = Measurement("start", targets=[2])
_, final_probabilities = final_measurement.measurement_comp_basis(state_final)

np.testing.assert_allclose(initial_probabilities, final_probabilities)